# 0. Installing packages

In [1]:
!pip uninstall scikit-learn scikit-survival -y

!pip install scikit-learn
!pip install scikit-survival

!pip install lifelines

!pip install joblib

!pip install xgboost

!pip install torchsurv

!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 268.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 322.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 204.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 250.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 298.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 316.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.7/396.7 kB 319.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 274.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 190.0 MB/s eta 0:00:0

In [2]:
import os 
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sksurv
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.metrics import log_loss
from sklearn.exceptions import FitFailedWarning
from sklearn.base import BaseEstimator

from torchsurv.loss.cox import neg_partial_log_likelihood
from torchsurv.metrics.cindex import ConcordanceIndex

import itertools
from itertools import product

import lifelines
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

from xgboost import XGBRegressor

import joblib

import warnings

import time

import torch

import lightgbm as lgb

# 1. Utils

## 1.1 Models

### 1.1.1 COXPH

In [3]:
def train_cox(train_data, train_labels, test_data, test_labels, verbose=True):
    
    """
    Trains a standard CoxPH model without regularization (minimal alpha for numerical stability).
    - Re-trains model on whole training split.
    - Evaluates performance on the test split via Harrel's C.
    - Returns final refit model, timing information, and c-index for evaluation.
    """

    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    # Step 1: Model fit
    if verbose:
        print("\nFitting the standard Cox Proportional Hazards model...")

    start_fit_time = time.time()
    cox_model = CoxPHSurvivalAnalysis(alpha=1e-6)
    cox_model.fit(train_data, labels_array)
    end_fit_time = time.time()
    fit_time = end_fit_time - start_fit_time

    if verbose:
        print(f"Fitting took {fit_time:.2f} seconds.")

    timing_info = {
        'fit_time': fit_time
    }

    # Step 2: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = cox_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return cox_model, timing_info, c_index

### 1.1.2 EN

In [4]:
def train_opt_EN(train_data, train_labels, test_data, test_labels,
                 l1_ratios=np.linspace(0.1, 1.0, 10), alphas=np.logspace(-3, 0, 5), max_iter=100, cv_folds=5, verbose=True):
    """
    EN model hyperparameter optimization:
        - Uses a fixed grid of alphas for tuning.
        - 5-fold CV along an alpha-lambda grid.
        - Determines optimal alpha and l1_ratio.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """

    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)
    
    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    # Step 1: Set up CV grid
    if verbose:
        print(f"Using fixed alpha grid ranging from {alphas.min():.5f} to {alphas.max():.5f} with 5 alphas.")

    param_grid = {
        'l1_ratio': l1_ratios,
        'alphas': [[alpha] for alpha in alphas]
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        CoxnetSurvivalAnalysis(max_iter=max_iter),
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=1 if verbose else 0
    )
    
    grid_search.fit(train_data, labels_array)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_ 
    best_l1_ratio = grid_search.best_params_['l1_ratio']
    best_alpha = grid_search.best_params_['alphas'][0]

    if verbose:
        print(f"\nBest l1_ratio: {best_l1_ratio:.2f}, Best alpha: {best_alpha:.5f}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")

    start_refit_time = time.time()
    final_model = CoxnetSurvivalAnalysis(l1_ratio=best_l1_ratio, alphas=[best_alpha], max_iter=max_iter)
    final_model.fit(train_data, labels_array)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)
    
    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


### 1.1.3 Ridge

In [5]:
def train_opt_ridge(train_data, train_labels, test_data, test_labels,
                    alphas=np.logspace(-3, 0, 5), cv_folds=5, verbose=True):
    """
    Ridge (Cox Proportional Hazards) model hyperparameter optimization:
        - Uses a fixed grid of alphas for tuning.
        - 5-fold CV along an alpha grid.
        - Determines optimal alpha.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """


    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)
    
    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])


    # Step 1: Set up CV grid
    if verbose:
        print(f"Using fixed alpha grid ranging from {alphas.min():.5f} to {alphas.max():.5f} with {len(alphas)} alphas.")

    param_grid = {
        'alpha': alphas
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        CoxPHSurvivalAnalysis(),
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=1 if verbose else 0
    )
    
    grid_search.fit(train_data, labels_array)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_
    best_alpha = grid_search.best_params_['alpha']

    if verbose:
        print(f"\nBest alpha: {best_alpha:.5f}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal alpha...")

    start_refit_time = time.time()
    final_model = CoxPHSurvivalAnalysis(alpha=best_alpha)
    final_model.fit(train_data, labels_array)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


### 1.1.4 LASSO

In [6]:
def train_opt_lasso(train_data, train_labels, test_data, test_labels,
                    alphas=np.logspace(-3, 0, 5), max_iter=100, cv_folds=5, verbose=True):
    """
    Lasso model hyperparameter optimization:
        - Uses a fixed grid of alphas for tuning.
        - 5-fold CV along an alpha grid.
        - Determines optimal alpha.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """

    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)

    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    # Step 1: Set up CV grid
    if verbose:
        print(f"Using fixed alpha grid ranging from {alphas.min():.5f} to {alphas.max():.5f} with {len(alphas)} alphas.")

    param_grid = {
        'alphas': [[alpha] for alpha in alphas]
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        CoxnetSurvivalAnalysis(l1_ratio=1.0, max_iter=max_iter),  # Set l1_ratio to 1.0 for Lasso
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=1 if verbose else 0
    )
    
    grid_search.fit(train_data, labels_array)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_
    best_alpha = grid_search.best_params_['alphas'][0]

    if verbose:
        print(f"\nBest alpha: {best_alpha:.5f}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal alpha...")

    start_refit_time = time.time()
    final_model = CoxnetSurvivalAnalysis(l1_ratio=1.0, alphas=[best_alpha], max_iter=max_iter)
    final_model.fit(train_data, labels_array)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


### 1.1.5 Random Survival Forests

In [7]:
def train_opt_RF(train_data, train_labels, test_data, test_labels,
                 n_estimators_range=[50, 100, 200], max_depth_range=[3, 7, 10], 
                 cv_folds=5, verbose=True):
    """
    Random Survival Forest (RSF) model hyperparameter optimization:
        - Uses a fixed grid of hyperparameters for tuning.
        - 5-fold CV along hyperparameter grid.
        - Determines optimal settings for n_estimators, max_depth, etc.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """

    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)

    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    # Step 1: Set up CV grid
    param_grid = {
        'n_estimators': n_estimators_range,
        'max_depth': max_depth_range
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        RandomSurvivalForest(min_samples_leaf = 10),
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=10 if verbose else 0,
        pre_dispatch = '2*n_jobs'
    )
    
    grid_search.fit(train_data, labels_array)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_
    best_n_estimators = grid_search.best_params_['n_estimators']
    best_max_depth = grid_search.best_params_['max_depth']

    if verbose:
        print(f"\nBest n_estimators: {best_n_estimators}, Best max_depth: {best_max_depth}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")

    start_refit_time = time.time()
    final_model = RandomSurvivalForest(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_leaf=10)
    final_model.fit(train_data, labels_array)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


#### 1.1.5.2 Ranger implementation via skranger

In [8]:
!pip install skranger

from skranger.ensemble import RangerForestSurvival
from sksurv.util import Surv
from sklearn.model_selection import KFold
from sksurv.metrics import concordance_index_censored

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.5 MB/s eta 0:00:00a 0:00:01m


In [9]:
def train_opt_RangerRF(train_data, train_labels, test_data, test_labels,
                     num_trees_range=[50, 100, 200],
                     max_depth_range=[3, 7, 10],
                     cv_folds=5, verbose=True):
    """
    Random Survival Forest (RSF) model hyperparameter optimization using skranger:
    - Performs grid search over num_trees and max_depth
    - Uses K-Fold cross-validation to determine best params
    - Re-trains the best model on the full training data
    - Evaluates on test data and returns results
    """

    warnings.filterwarnings("ignore", category=UserWarning)

    T_train = train_labels.iloc[:, 1].values  # Time-to-event
    E_train = train_labels.iloc[:, 0].values  # Event indicator
    y_train_structured = Surv.from_arrays(event=E_train, time=T_train)

    # Step 1: Set up CV grid
    param_grid = [(num_trees, max_depth) for num_trees in num_trees_range for max_depth in max_depth_range]
    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    best_c_index = -np.inf
    best_params = param_grid[0]
    all_cv_results = []
    
    # Step 2: Hyperparameter optimisation
    for num_trees, max_depth in param_grid:
        if verbose:
            print(f"\nTesting num_trees={num_trees}, max_depth={max_depth}...")
        
        c_indices = []
        start_cv_time = time.time()

        for train_idx, val_idx in cv.split(train_data):
            train_X, val_X = train_data.iloc[train_idx], train_data.iloc[val_idx]
            train_y, val_y = y_train_structured[train_idx], y_train_structured[val_idx]

            rsf = RangerForestSurvival(n_estimators=num_trees, max_depth=max_depth, min_node_size=20, seed = 42, oob_error = False)
            rsf.fit(train_X, train_y)
            
            predictions = rsf.predict(val_X)
            c_index = concordance_index_censored(val_y['event'], val_y['time'], predictions)[0]
            c_indices.append(c_index)

        mean_c_index = np.mean(c_indices)
        end_cv_time = time.time()

        all_cv_results.append({
            'num_trees': num_trees,
            'max_depth': max_depth,
            'mean_c_index': mean_c_index,
            'cv_time': end_cv_time - start_cv_time
        })

        if mean_c_index > best_c_index:
            best_c_index = mean_c_index
            best_params = (num_trees, max_depth)

    cv_results = pd.DataFrame(all_cv_results)

    # Step 3: Model fit
    best_num_trees, best_max_depth = best_params
    if verbose:
        print(f"\nBest num_trees: {best_num_trees}, Best max_depth: {best_max_depth}")

    start_refit_time = time.time()
    final_model = RangerForestSurvival(n_estimators=best_num_trees, max_depth=best_max_depth, min_node_size=20, seed = 42, oob_error = False)
    final_model.fit(train_data, y_train_structured)
    end_refit_time = time.time()

    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    # Step 4: Harrel's C on the test split
    T_test = test_labels.iloc[:, 1].values
    E_test = test_labels.iloc[:, 0].values
    y_test_structured = Surv.from_arrays(event=E_test, time=T_test)

    test_predictions = final_model.predict(test_data)
    c_index = concordance_index_censored(y_test_structured['event'], y_test_structured['time'], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    timing_info = {'refit_time': refit_time}

    return final_model, cv_results, timing_info, c_index


### 1.1.6 GBM

#### 1.1.6.1 LightGBM

In [10]:
def cox_ph_loss_lgb(preds, dataset):
    labels = dataset.get_label()
    survival_time = np.abs(labels)
    event = (labels > 0).astype(int)

    #sort by survival time
    sorted_indices = np.argsort(survival_time)
    preds = preds[sorted_indices]
    event = event[sorted_indices]
    survival_time = survival_time[sorted_indices]

    #compute risk set using Breslow's method for ties
    exp_preds = np.exp(preds)
    risk_set_sum = np.zeros_like(exp_preds)
    accumulated_sum = 0.0
    last_time = None
    last_exp_p = 0.0

    for i in reversed(range(len(preds))):
        if last_time is None or survival_time[i] < last_time:
            accumulated_sum += last_exp_p 
            last_exp_p = 0.0  
        last_exp_p += exp_preds[i] 
        risk_set_sum[i] = accumulated_sum + last_exp_p 
        last_time = survival_time[i]

    grad = np.zeros_like(preds)
    hess = np.zeros_like(preds)

    r_k = 0.0
    s_k = 0.0
    for i in range(len(preds)):
        if event[i] == 1:
            r_k += 1.0 / max(risk_set_sum[i], 1e-10)
            s_k += 1.0 / max(risk_set_sum[i] ** 2, 1e-10)

        exp_p = exp_preds[i]
        grad[i] = exp_p * r_k - event[i]
        hess[i] = exp_p * (r_k - exp_p * s_k)

    #map back to original order
    grad_unsorted = np.zeros_like(grad)
    hess_unsorted = np.zeros_like(hess)
    grad_unsorted[sorted_indices] = grad
    hess_unsorted[sorted_indices] = hess

    return grad_unsorted, hess_unsorted


In [11]:
def cindex_metric(preds, dataset):
    """
    Harrel's C evaluation metric for LightGBM
    
    args:
        preds (np.ndarray): Predicted risk scores 
        dataset (lightgbm.Dataset): LightGBM dataset containing labels.

    returns:
        tuple: (metric name, metric value, is_higher_better)
    """

    
    labels = dataset.get_label()
    
    survival_time = np.abs(labels) 
    event = (labels > 0).astype(int)  
    
    cindex = concordance_index_censored(event.astype(bool), survival_time, preds)[0]
    
    return 'cindex', cindex, True 


In [12]:
def train_opt_lightGBM(
    train_data, train_labels, test_data, test_labels,
    n_estimators_range=[50, 100, 200],
    num_leaves_range=[7, 127, 1023],
    cv_folds=5, verbose=True
):
    """
    Optimized LightGBM Cox model hyperparameter tuning:
        - Tests a hyperparameter grid in a CV approach.
        - Records results for all tested combinations and steps.
    """

    #encode survival times for compativility with LightGBM: Positive for events, Negative for censoring
    y_train_encoded = np.array([
        survival_time if event == 1 else -survival_time
        for survival_time, event in zip(train_labels[f"{endpoint}_followup"], train_labels[f"{endpoint}_status"])
    ])
    y_test_encoded = np.array([
        survival_time if event == 1 else -survival_time
        for survival_time, event in zip(test_labels[f"{endpoint}_followup"], test_labels[f"{endpoint}_status"])
    ])

    dtrain = lgb.Dataset(
        train_data,
        label=y_train_encoded,
        free_raw_data=False
    )

    # Step 1: Set up CV grid
    param_grid = product(
        n_estimators_range,
        num_leaves_range,
    )

    best_params = None
    best_c_index = -np.inf
    best_std = None
    best_model = None
    cv_times = []

    all_cv_results = []
    

    # Step 2: Hyperparameter optimisation
    for n_estimators, num_leaves in param_grid:
        
        if verbose:
            print(f"\nTesting n_estimators={n_estimators}, num_leaves={num_leaves}...")
            
        params = {
            'objective': cox_ph_loss_lgb,
            'boosting_type': 'gbdt',
            'learning_rate': 0.1,
            'num_leaves': num_leaves,
            'min_data_in_leaf': 10,
            'max_depth': -1,  # allow unrestricted depth as tuned by num_leaves
            'metric': 'None',
            'seed': 42,
            'verbose': -1
        }

        start_cv_time = time.time()
        
        cv_results = lgb.cv(
            params=params,
            train_set=dtrain,
            num_boost_round=n_estimators,
            nfold=cv_folds,
            stratified=False,
            shuffle=True,
            metrics=None,
            feval=cindex_metric,
            seed=42
        )
        cv_time = time.time() - start_cv_time
        cv_times.append(cv_time)

        mean_cindex = cv_results['valid cindex-mean'][-1]
        std_cindex = cv_results['valid cindex-stdv'][-1]

        all_cv_results.append({
            'n_estimators': n_estimators,
            'num_leaves': num_leaves,
            'mean_cindex': mean_cindex,
            'std_cindex': std_cindex,
            'cv_time': cv_time
        })

        if mean_cindex > best_c_index:
            best_c_index = mean_cindex
            best_std = std_cindex
            best_params = params
            best_params['n_estimators'] = n_estimators  # Add n_estimators to best_params

    cv_results_df = pd.DataFrame(all_cv_results)

    if verbose:
        print(f"\nBest parameters: {best_params}")
        print(f"Mean CV C-index: {best_c_index:.4f} ± {best_std:.4f}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")
    start_refit_time = time.time()
    final_model = lgb.train(
        params=best_params,
        train_set=dtrain,
        num_boost_round=best_params['n_estimators']
    )
    refit_time = time.time() - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    timing_info = {
        'cv_time': sum(cv_times),
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    test_predictions = final_model.predict(test_data)
    y_event_test = (y_test_encoded > 0).astype(int)
    y_time_test = np.abs(y_test_encoded)
    c_index = concordance_index_censored(
        y_event_test.astype(bool),
        y_time_test,
        test_predictions
    )[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results_df, timing_info, c_index


#### 1.1.6.2 sksurv

In [13]:
def train_opt_GBM(train_data, train_labels, test_data, test_labels,
                  n_estimators_range=[50, 100, 200],
                  max_depth_range=[3, 7, 10],
                  cv_folds=5, verbose=True):
    
    """
    Gradient Boosting Machine (GBM) model hyperparameter optimization for survival analysis:
        - Uses a fixed grid of hyperparameters for tuning.
        - 5-fold CV along hyperparameter grid.
        - Determines optimal settings for learning_rate, n_estimators, max_depth, etc.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """

    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)

    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    
    # Step 1: Set up CV grid
    param_grid = {
        'n_estimators': n_estimators_range,
        'max_depth': max_depth_range,
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        GradientBoostingSurvivalAnalysis(learning_rate = 0.1, min_samples_leaf = 10),
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=10 if verbose else 0,
        pre_dispatch='2*n_jobs'
    )

    grid_search.fit(train_data, labels_array)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_
    best_n_estimators = grid_search.best_params_['n_estimators']
    best_max_depth = grid_search.best_params_['max_depth']

    if verbose:
        print(f"\nBest n_estimators: {best_n_estimators}, "
              f"Best max_depth: {best_max_depth}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")

    start_refit_time = time.time()
    final_model = GradientBoostingSurvivalAnalysis(
        learning_rate=0.1,
        n_estimators=best_n_estimators,
        max_depth=best_max_depth,
        min_samples_leaf=10
    )
    final_model.fit(train_data, labels_array)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


### 1.1.7 XGB

In [14]:
def train_opt_XGB(train_data, train_labels, test_data, test_labels,
                  n_estimators_range=[50, 100, 200],
                  max_depth_range=[3, 7, 10],
                  cv_folds=5, verbose=True):
    """
    XGBoost (XGB) Cox model hyperparameter optimization for survival analysis:
        - Uses a fixed grid of hyperparameters for tuning.
        - 5-fold CV along hyperparameter grid.
        - Determines optimal settings for learning_rate, n_estimators, max_depth, etc.
        - Re-trains model on the whole training split using optimal settings.
        - Evaluates performance on the test split.
        - Returns final refit model, CV results, timing information, and c-index for evaluation.
    """

    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)

    labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                            dtype=[('event', '?'), ('time', '<f8')])

    
    # Step 1: Set up CV grid
    param_grid = {
        'n_estimators': n_estimators_range,
        'max_depth': max_depth_range
    }

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")

    start_cv_time = time.time()
    grid_search = GridSearchCV(
        XGBRegressor(objective='survival:cox', eval_metric='cox-nloglik', tree_method='exact', 
                     learning_rate = 0.1, min_child_weight=10),
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        verbose=10 if verbose else 0,
        pre_dispatch='2*n_jobs'
    )

    grid_search.fit(train_data, labels_array['time'])
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    best_model = grid_search.best_estimator_
    best_n_estimators = grid_search.best_params_['n_estimators']
    best_max_depth = grid_search.best_params_['max_depth']

    if verbose:
        print(f"\nBest n_estimators: {best_n_estimators}, "
              f"Best max_depth: {best_max_depth}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")

    start_refit_time = time.time()
    final_model = XGBRegressor(
        objective='survival:cox',
        eval_metric='cox-nloglik',
        tree_method='exact',
        learning_rate=0.1,
        min_child_weight=10,
        n_estimators=best_n_estimators,
        max_depth=best_max_depth
    )
    final_model.fit(train_data, labels_array['time'])
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    test_labels_array = np.array([(status, time) for status, time in zip(test_labels.iloc[:, 0], test_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])

    test_predictions = final_model.predict(test_data)

    c_index = concordance_index_censored(test_labels_array["event"], test_labels_array["time"], test_predictions)[0]

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


### 1.1.8 DL

In [15]:
class DeepSurvWrapper(BaseEstimator):
    def __init__(self, optimizer, input_dim, epochs=50, batch_size=32, lr=0.001, dropout=0.2, layer_sizes=[32,32], verbose=True):
        
        """
        Wrapper for integrating DeepSurv with sklearn's GridSearchCV.

        Parameters:
        - model: A PyTorch DeepSurv model instance.
        - optimizer: A PyTorch optimizer class (e.g., torch.optim.Adam).
        - epochs: Number of training epochs.
        - batch_size: Batch size for training.
        - lr: Learning rate for the optimizer.
        - layer_sizes: Sizes of the layers (and thus also number of layers)
        - verbose: Whether to print training progress.
        """

        self.input_dim = input_dim
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.verbose = verbose
        self.dropout = dropout
        self.layer_sizes = layer_sizes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
        self.model = DeepSurvModel(self.input_dim, self.layer_sizes, self.dropout).to(self.device)  
        torch.set_num_threads(24)

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def get_params(self, deep=True):
        return {
            "input_dim": self.input_dim,
            "optimizer": self.optimizer,
            "epochs": self.epochs,
            "batch_size": self.batch_size,
            "lr": self.lr,
            "verbose": self.verbose,
            "dropout": self.dropout,
            "layer_sizes": self.layer_sizes
        }

    def fit(self, X, y, patience=5):
        
        """
        Train the DeepSurv model.

        Parameters:
        - X: Features (numpy array or pandas DataFrame).
        - y: Targets as a DataFrame with columns ["CVD_status", "CVD_followup"].
          - "CVD_status": Binary array indicating whether the event occurred.
          - "CVD_followup": Array of survival times (or censoring times).
        - patience: Number of epochs to wait for improvement before stopping.
        """

        #convert x and y to tensors
        self.model.train()
        X_tensor = torch.tensor(X.values, dtype=torch.float32).to(self.device)
        event_tensor = torch.tensor(y.iloc[:,0].values, dtype=torch.float32).to(self.device)  # Event (0/1)
        time_tensor = torch.tensor(y.iloc[:,1].values, dtype=torch.float32).to(self.device)  # Survival time

        #optimizer and loss function
        optimizer = self.optimizer(self.model.parameters(), lr=self.lr)

        best_train_loss = float('inf')
        epochs_without_improvement = 0
        
        num_samples = X_tensor.size(0)
        num_batches = int(np.ceil(num_samples / self.batch_size))


        #training loop
        for epoch in range(self.epochs):
            epoch_loss = 0.0

            #shuffle the data at the beginning of each epoch
            permutation = torch.randperm(num_samples)
            X_shuffled = X_tensor[permutation]
            event_shuffled = event_tensor[permutation]
            time_shuffled = time_tensor[permutation]

            for batch_idx in range(num_batches):
                start_idx = batch_idx * self.batch_size
                end_idx = min(start_idx + self.batch_size, num_samples)

                batch_X = X_shuffled[start_idx:end_idx]
                batch_event = event_shuffled[start_idx:end_idx]
                batch_time = time_shuffled[start_idx:end_idx]

                optimizer.zero_grad()
                predictions = self.model(batch_X).squeeze()
                loss = neg_partial_log_likelihood(predictions, batch_event.bool(), batch_time)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item() * batch_X.size(0)

            #check for improvement on training loss
            avg_epoch_loss = epoch_loss / num_samples

            if self.verbose:
                print(f"Epoch {epoch+1}/{self.epochs}, Loss: {avg_epoch_loss:.4f}")

            if avg_epoch_loss < best_train_loss:
                best_train_loss = avg_epoch_loss
                epochs_without_improvement = 0
                #save the best model
                best_model_state = self.model.state_dict()
            else:
                epochs_without_improvement += 1
    
            #early stopping
            if epochs_without_improvement >= patience:
                if self.verbose:
                    print("Early stopping due to no improvement in training loss")
                break
    
        #optionally load the weights of the best model
        if 'best_model_state' in locals():
            self.model.load_state_dict(best_model_state)

    def predict(self, X):
        """
        Predict risk scores for the given features.

        Parameters:
        - X: Features (numpy array or pandas DataFrame).

        Returns:
        - Risk scores (numpy array).
        """
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X.values, dtype=torch.float32).to(self.device)
            risk_scores = self.model(X_tensor).squeeze().cpu().numpy()  # Squeeze to reduce unnecessary dimensions
        return risk_scores

    def score(self, X, y):
        """
        Compute the concordance index as the evaluation metric.

        Parameters:
        - X: Features (numpy array or pandas DataFrame).
        - y: Targets as a DataFrame with columns ["CVD_status", "CVD_followup"].

        Returns:
        - Concordance index score (float).
        """
        risk_scores = self.predict(X)
        risk_scores_tensor = torch.tensor(risk_scores, dtype=torch.float32).to(self.device)
        
        #prepare event and time tensors
        event_tensor = torch.tensor(y.iloc[:,0].values, dtype=torch.bool).to(self.device)  # Event (0/1)
        time_tensor = torch.tensor(y.iloc[:,1].values, dtype=torch.float32).to(self.device)  # Survival time

        cox_index = ConcordanceIndex()
        c_index = cox_index(risk_scores_tensor, event_tensor, time_tensor)

        return c_index


#define model architecture
class DeepSurvModel(torch.nn.Module):
    def __init__(self, input_dim, layer_sizes, dropout):
        """
        Args:
            input_dim (int): Number of input features.
            layer_sizes (list): List of integers where each integer specifies 
                                the number of neurons in a layer.
            dropout (float): Dropout probability.
        """
        super(DeepSurvModel, self).__init__()
        
        self.layers = torch.nn.ModuleList() 
        self.batch_norms = torch.nn.ModuleList()  # BatchNorm for each layer
        
        #add first batch normalization layer for input
        self.batch_norms.append(torch.nn.BatchNorm1d(input_dim))
        
        #dynamically add linear, activation, and dropout layers
        prev_dim = input_dim
        for layer_size in layer_sizes:
            self.layers.append(torch.nn.Linear(prev_dim, layer_size))
            self.batch_norms.append(torch.nn.BatchNorm1d(layer_size))  #BatchNorm after each Linear layer
            prev_dim = layer_size
        
        #output layer for final prediction
        self.output = torch.nn.Linear(prev_dim, 1)  # Estimating log hazards
        
        #activation and dropout
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        #apply batch norm to input
        x = self.batch_norms[0](x)
        
        #pass through each linear layer with ReLU and Dropout
        for i, layer in enumerate(self.layers):
            x = layer(x)
            x = self.batch_norms[i + 1](x)  # BatchNorm after linear
            x = self.relu(x)
            x = self.dropout(x)
        
        #output layer
        x = self.output(x)
        return x

In [16]:
def train_opt_DL(
    train_data, train_labels, test_data, test_labels,
    learning_rate_range=[0.001],
    batch_size_range=[50000],
    epoch_range=[100],
    dropout_range=[0.2],
    layer_numbers=[2, 3, 5],
    layer_sizes=[16, 64, 256],
    cv_folds=5, verbose=True
):
    """
    Optimized deep learning survival model hyperparameter tuning:
        - Dynamically generates all combinations of uniform layer sizes for the specified number of layers.
        - Uses GridSearchCV for hyperparameter optimization.
        - Evaluates using concordance index (c-index).
    """

    
    
    warnings.simplefilter("ignore", DeprecationWarning)
    
    ## Step 1: Set up CV grid
    # Generate all possible uniform layer configurations
    layer_configurations = [
        [size] * num_layers
        for num_layers in layer_numbers
        for size in layer_sizes
    ]

    param_grid = {
        'optimizer': [torch.optim.Adam],
        'lr': learning_rate_range,
        'batch_size': batch_size_range,
        'epochs': epoch_range,
        'dropouts': dropout_range,
        'layer_sizes': layer_configurations
    }

    # Step 2: Hyperparameter optimisation
    if verbose:
        print("Starting hyperparameter optimization using GridSearchCV...")
    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    start_cv_time = time.time()
    deep_surv_wrapper = DeepSurvWrapper(
        optimizer=torch.optim.Adam,
        epochs=5,  # Placeholder, will be updated by GridSearchCV
        batch_size=32,  # Placeholder
        lr=1e-3,  # Placeholder,
        dropout=0.1,  # Placeholder
        verbose=verbose,
        layer_sizes=[32, 32],  # Placeholder
        input_dim=train_data.shape[1]
    )
    grid_search = GridSearchCV(
        estimator=deep_surv_wrapper,
        param_grid=param_grid,
        cv=cv,
        n_jobs=1,
        verbose=10 if verbose else 0
    )
    grid_search.fit(train_data, train_labels)
    end_cv_time = time.time()
    cv_time = end_cv_time - start_cv_time

    if verbose:
        print(f"Hyperparameter optimization took {cv_time:.2f} seconds.")

    #get best model and parameters 
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    if verbose:
        print("\nBest Parameters:")
        for param, value in best_params.items():
            print(f"{param}: {value}")

    # Step 3: Model fit
    if verbose:
        print("\nRefitting the model on the entire training set with optimal parameters...")

    start_refit_time = time.time()
    final_model = grid_search.best_estimator_
    final_model.fit(train_data, train_labels)
    end_refit_time = time.time()
    refit_time = end_refit_time - start_refit_time

    if verbose:
        print(f"Refitting took {refit_time:.2f} seconds.")

    cv_results = pd.DataFrame(grid_search.cv_results_)

    timing_info = {
        'cv_time': cv_time,
        'refit_time': refit_time
    }

    # Step 4: Harrel's C on the test split
    if verbose:
        print("\nEvaluating model performance on the test split...")

    #predict
    test_predictions = final_model.predict(test_data)
    test_predictions_tensor = torch.tensor(test_predictions, dtype=torch.float32)
    event_tensor = torch.tensor(test_labels.iloc[:,0].values, dtype=torch.bool)  # Event (0/1)
    time_tensor = torch.tensor(test_labels.iloc[:,1].values, dtype=torch.float32)  # Survival time
    
    #c-index
    cox_index = ConcordanceIndex()
    c_index = cox_index(test_predictions_tensor, event_tensor, time_tensor)

    if verbose:
        print(f"Concordance index (c-index) on test split: {c_index:.4f}")

    return final_model, cv_results, timing_info, c_index


## 1.2 Misc

### 1.2.1 Data Prep

In [17]:
def load_and_prepare_data(file, endpoint, sample_size=None):
    """
    Function to load the imputed dataset, filter by endpoint-specific rules, and prepare labels.
    
    Args:
    - file (str): The file path to the imputed data split.
    - endpoint (str): The endpoint to use for filtering data.
    - sample_size (int, optional): Number of samples to retain. If None, use full dataset.

    Returns:
    - df_filtered (DataFrame): The filtered dataset.
    - labels (DataFrame): The corresponding labels for modeling.
    """
    # Download the imputed dataset
    dl_cmd = f"dx download '{file}' --overwrite"
    !{dl_cmd}

    # Load the dataset
    df = pd.read_csv(file.split('/')[-1], sep="\t")

    # Round continuous variables to 1 decimal place
    continuous_cols = df.select_dtypes(include=[np.number]).columns
    df[continuous_cols] = df[continuous_cols].round(2)

    # Endpoint specific exclusion - baseline endpoint status
    eids_to_include = df[df[f"{endpoint}_at_base"] == False]["eid"]
    df_filtered = df[df["eid"].isin(eids_to_include)]

    # Sex-based exclusion
    if endpoint == "PC":
        eids_to_exclude = df[df["clinicalrisk_Sex_0"] == True]["eid"]
        df_filtered = df_filtered[~df_filtered["eid"].isin(eids_to_exclude)]
    elif endpoint == "BC":
        eids_to_exclude = df[df["clinicalrisk_Sex_1"] == True]["eid"]
        df_filtered = df_filtered[~df_filtered["eid"].isin(eids_to_exclude)]

    if sample_size and len(df_filtered) > sample_size:
        df_filtered = df_filtered.sample(n=sample_size, random_state=42)

    labels = df_filtered[[f"{endpoint}_status", f"{endpoint}_followup", "eid", "testtrain"]].copy()
    labels = labels.set_index("eid")

    return df_filtered, labels

In [18]:
def split_train_test(df_filtered, labels, testtrain_column='testtrain'):
    
    train_data = df_filtered[df_filtered[testtrain_column] == 'train'].drop(columns=[testtrain_column])
    test_data = df_filtered[df_filtered[testtrain_column] == 'test'].drop(columns=[testtrain_column])

    train_labels = labels[labels[testtrain_column] == 'train'].drop(columns=[testtrain_column])
    test_labels = labels[labels[testtrain_column] == 'test'].drop(columns=[testtrain_column])

    return train_data, test_data, train_labels, test_labels

In [19]:
def remove_low_variance_features(train_data, test_data, threshold=0.01, verbose=True):
    """
    Remove low variance and highly correlated features from train and test datasets.

    Parameters:
    - train_data (DataFrame): Training dataset.
    - test_data (DataFrame): Test dataset.
    - threshold (float): Threshold for variance below which features will be removed.
    - verbose (bool): If True, prints out information about removed features.

    Returns:
    - train_data_reduced (DataFrame): Reduced training dataset.
    - test_data_reduced (DataFrame): Reduced test dataset.
    - retained_columns (Index): Names of the retained columns.
    """
    if verbose:
        print("Separating boolean and numeric data...")

    # Separate boolean and numeric columns
    bool_columns = train_data.select_dtypes(include=['bool']).columns
    numeric_columns = train_data.select_dtypes(include=[np.number]).columns

    # Ensure no unsupported types are present
    unsupported_columns = train_data.columns.difference(bool_columns.union(numeric_columns))
    if not unsupported_columns.empty:
        raise ValueError(f"Unsupported data types found in columns: {list(unsupported_columns)}")

    if verbose:
        print(f"Boolean columns: {len(bool_columns)}")
        print(f"Numeric columns: {len(numeric_columns)}")

    if verbose:
        print("Checking for missing or infinite values in numeric columns...")

    # Check for missing or infinite values in numeric columns only
    assert not train_data[numeric_columns].isnull().any().any(), "train_data contains NaN values in numeric columns"
    assert not np.isinf(train_data[numeric_columns].values).any(), "train_data contains infinite values in numeric columns"
    assert not test_data[numeric_columns].isnull().any().any(), "test_data contains NaN values in numeric columns"
    assert not np.isinf(test_data[numeric_columns].values).any(), "test_data contains infinite values in numeric columns"

    if verbose:
        print("Removing low variance features...")

    # Remove low variance features (applies to all columns together)
    selector = VarianceThreshold(threshold=threshold)
    train_data_reduced = selector.fit_transform(train_data)
    test_data_reduced = selector.transform(test_data)
    retained_columns = train_data.columns[selector.get_support(indices=True)]

    # Convert reduced numpy arrays back to DataFrame
    train_data_reduced = pd.DataFrame(train_data_reduced, columns=retained_columns, index=train_data.index)
    test_data_reduced = pd.DataFrame(test_data_reduced, columns=retained_columns, index=test_data.index)

    if verbose:
        print(f"Number of low variance features removed: {train_data.shape[1] - len(retained_columns)}")
        print(f"Number of features retained: {len(retained_columns)}")

    if verbose:
        print("Removing highly correlated features from numeric columns...")

    # Remove highly correlated features (applies only to numeric columns)
    correlation_matrix = train_data_reduced[numeric_columns].corr()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(abs(upper_triangle[column]) > 0.99)]

    train_data_reduced = train_data_reduced.drop(columns=to_drop, errors='ignore')
    test_data_reduced = test_data_reduced.drop(columns=to_drop, errors='ignore')
    retained_columns = train_data_reduced.columns

    if verbose:
        print(f"Number of highly correlated features removed: {len(to_drop)}")
        print(f"Number of features retained after correlation filtering: {len(retained_columns)}")

    return train_data_reduced, test_data_reduced, retained_columns


In [20]:
"""
endpoint_names = [
    "CVD", "HF", "BC", "DM", "LD", "RD", "AF",  "CAD", "VT", "ISS", "AAA", "PAD", 
    "AS", "COPD", "LC", "MEL", "CRC", "PC", "PD", "OP", "CAT", "POAG", "HT", "AD" 
]
"""

endpoint_names = [
   "AD", "BC", "CVD" 
]

print(endpoint_names)


['AD', 'BC', 'CVD']


### 1.2.2 Saving & Uploading

In [21]:
def upload_model_cvresults_timing(model, model_type, endpoint, combo_name, cvresults, timing, cv_split, sample_size, directory="Benchmarking/ResultsScaling"):
    # Filenames for model, cross-validation results, and timing results
    filename_model = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}.pkl"
    filename_timings = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_timings.tsv"
    
    # Save model
    joblib.dump(model, filename_model)
    upload_cmd_model = f"dx upload {filename_model} --path {directory}/{filename_model}"
    !{upload_cmd_model}
    
    # Save cross-validation results if they exist
    if cvresults is not None:
        filename_cvresults = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_cvresults.tsv"
        cvresults.to_csv(filename_cvresults, sep='\t', index=False)
        upload_cmd_cvresults = f"dx upload {filename_cvresults} --path {directory}/{filename_cvresults}"
        !{upload_cmd_cvresults}
        os.remove(filename_cvresults)

    # Save timing information
    timing_df = pd.DataFrame([timing])
    timing_df.to_csv(filename_timings, sep='\t', index=False)
    upload_cmd_timings = f"dx upload {filename_timings} --path {directory}/{filename_timings}"
    !{upload_cmd_timings}

    os.remove(filename_model)
    os.remove(filename_timings)

In [22]:
def save_and_upload_lps_cindex(model, model_type, train_data, test_data, train_labels, test_labels, c_index, endpoint, combo_name, cv_split, sample_size, directory="Benchmarking/ResultsScaling"):
    """
    Save and upload linear predictors and concordance index for the Cox, LightGBM, and RangerRF models.
    """
    if model_type == "lightGBM":
        retained_columns = model.feature_name()
    elif model_type in ["DL", "RangerRF"]:
        retained_columns = train_data.columns
    else:
        retained_columns = model.feature_names_in_

    train_data_reduced = train_data[retained_columns]
    test_data_reduced = test_data[retained_columns]

    train_lp = model.predict(train_data_reduced)
    test_lp = model.predict(test_data_reduced)
    
    train_lp_df = pd.DataFrame({"eid": train_labels.index, "LP": train_lp})
    test_lp_df = pd.DataFrame({"eid": test_labels.index, "LP": test_lp})

    train_lp_filename = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_train_LP.tsv"
    test_lp_filename = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_test_LP.tsv"
    train_lp_df.to_csv(train_lp_filename, sep='\t', index=False)
    test_lp_df.to_csv(test_lp_filename, sep='\t', index=False)
    
    performance_filename = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_performance.tsv"
    performance_df = pd.DataFrame([{'cv_split': cv_split, 'c_index': c_index}])
    performance_df.to_csv(performance_filename, sep='\t', index=False)

    upload_cmd_trainlp = f"dx upload {train_lp_filename} --path {directory}/{train_lp_filename}"
    upload_cmd_testlp = f"dx upload {test_lp_filename} --path {directory}/{test_lp_filename}"
    upload_cmd_performance = f"dx upload {performance_filename} --path {directory}/{performance_filename}"

    !{upload_cmd_trainlp}
    !{upload_cmd_testlp}
    !{upload_cmd_performance}

    os.remove(train_lp_filename)
    os.remove(test_lp_filename)
    os.remove(performance_filename)

In [23]:
def save_and_upload_feature_importance(model, model_type, train_data, train_labels, endpoint, combo_name, cv_split, sample_size, directory="Benchmarking/ResultsScaling"):
    """
    Save and upload feature importance or model coefficients for the trained model.
    """
    retained_columns = train_data.columns

    if model_type == "RangerRF":
        train_labels_array = np.array([(status, time) for status, time in zip(train_labels.iloc[:, 0], train_labels.iloc[:, 1])],
                                 dtype=[('event', '?'), ('time', '<f8')])
        result = permutation_importance(model, 
                                        train_data, train_labels_array, 
                                        n_repeats=1, 
                                        random_state=42, 
                                        n_jobs=16)
        coef_df = pd.DataFrame({
            "importance_mean": result.importances_mean,
            "importance_std": result.importances_std
        }, index=retained_columns).sort_values(by="importance_mean", ascending=False)
    elif model_type in ["EN", "Lasso", "Ridge", "Cox"]:
        coef_df = pd.DataFrame(model.coef_, index=retained_columns, columns=["Coefficient"])
    else:
        raise ValueError("Unsupported model type.")
        
    coeff_filename = f"{model_type}_{endpoint}_{combo_name}_cvsplit_{cv_split}_n{sample_size}_feature_importance.tsv"
    coef_df.to_csv(coeff_filename, sep='\t')

    upload_cmd_coef = f"dx upload {coeff_filename} --path {directory}/{coeff_filename}"
    os.system(upload_cmd_coef)

    os.remove(coeff_filename)

### 1.2.3 Predictor combos

In [24]:
always_include = ["clinicalrisk_Age.at.recruitment", "clinicalrisk_Sex_1", "eid", "testtrain"]

predictor_combinations = {
    "prs_metabolomics": ["prs_", "metabolomics_"],
    "pmh": ["pmh_"],
    #"ts": ["ts_"],
    #"metabolomics": ["metabolomics_"],
    #"prs": ["prs_"],
    "clinicalrisk": ["clinicalrisk_"],
    #"pmh_ts": ["pmh_", "ts_"],
    #"prs_metabolomics_pmh_ts": ["prs_", "metabolomics_", "pmh_", "ts_"],
    #"clinicalrisk_pmh_ts": ["clinicalrisk_", "pmh_", "ts_"],
    #"clinicalrisk_prs_metabolomics": ["clinicalrisk_", "prs_", "metabolomics_"],
    "everything": ["clinicalrisk_", "pmh_", "prs_", "metabolomics_"],
    #"score": ["score_"],
    #"qrisk": ["qrisk_"],
    #"prevent": ["prevent_"],
    "agesex": []
}


# 2. Final Loop

## 2.1 for everything

In [25]:
'''
Everything was conducted on mem3_ssd1_v2_x32 instances
'''

'\nEverything was conducted on mem3_ssd1_v2_x32 instances\n'

In [ ]:
cv_splits = [1,2,3,4,5]  # Specify the actual CV splits to process
processed_files = [f"Benchmarking/Processed/Processed_final_split_{i}_16012024.tsv" for i in cv_splits]
sample_sizes = [5000, 10000, 20000, 50000]
model_types = ["Cox", "EN", "Lasso", "Ridge", "GBM", "lightGBM", "DL", "RangerRF", "XGB"]
model_types = ["RangerRF"]


for cv_split, file in zip(cv_splits, processed_files):
    print(f"Starting CV Split: {cv_split}")

    for endpoint in endpoint_names:
        print(f"Processing Endpoint: {endpoint}")

        for sample_size in sample_sizes:
            df_filtered, labels = load_and_prepare_data(file, endpoint, sample_size)
            print(f"Processed {len(df_filtered)} rows for testing.")

            for combo_name, prefixes in predictor_combinations.items():
                print(f"Analyzing Combination: {combo_name}")
                if combo_name not in ["everything"]: continue
                
                selected_cols = always_include + [
                    col for col in df_filtered.columns
                    if any(col.startswith(prefix) for prefix in prefixes) and col not in always_include
                ]
                df_filtered2 = df_filtered[selected_cols]
                df_filtered2 = df_filtered2.set_index("eid").replace({'TRUE': 1, 'FALSE': 0})

                train_data, test_data, train_labels, test_labels = split_train_test(df_filtered2, labels)
                train_data, test_data, retained_columns = remove_low_variance_features(train_data, test_data, threshold=0.01)
                
                for model_type in model_types:
                    print(f"Training model: {model_type}")

                    if model_type == "EN":
                        best_model, results_df, timing, cindex = train_opt_EN(train_data, train_labels, test_data, test_labels)
                    elif model_type == "Lasso":
                        best_model, results_df, timing, cindex = train_opt_lasso(train_data, train_labels, test_data, test_labels)
                    elif model_type == "Ridge":
                        best_model, results_df, timing, cindex = train_opt_ridge(train_data, train_labels, test_data, test_labels)
                    elif model_type == "Cox":
                        best_model, timing, cindex = train_cox(train_data, train_labels, test_data, test_labels)
                        results_df = None
                    elif model_type == "RF":
                        best_model, results_df, timing, cindex = train_opt_RF(train_data, train_labels, test_data, test_labels)
                    elif model_type == "GBM":
                        best_model, results_df, timing, cindex = train_opt_GBM(train_data, train_labels, test_data, test_labels)
                    elif model_type == "lightGBM":
                        best_model, results_df, timing, cindex = train_opt_lightGBM(train_data, train_labels, test_data, test_labels)
                    elif model_type == "DL":
                        best_model, results_df, timing, cindex = train_opt_DL(train_data, train_labels, test_data, test_labels)
                    elif model_type == "RangerRF":
                        best_model, results_df, timing, cindex = train_opt_RangerRF(train_data, train_labels, test_data, test_labels)
                    elif model_type == "XGB":
                        best_model, results_df, timing, cindex = train_opt_XGB(train_data, train_labels, test_data, test_labels)
                    else:
                        raise ValueError("Unsupported model type.")

                    upload_model_cvresults_timing(best_model, model_type, endpoint, combo_name, results_df, timing, cv_split, sample_size)
                    save_and_upload_lps_cindex(best_model, model_type, train_data, test_data, train_labels, test_labels, cindex, endpoint, combo_name, cv_split, sample_size)

print("All tasks completed successfully.")

Starting CV Split: 1
Processing Endpoint: AD
[===========================================================>] Completed 1,626,953,279 of 1,626,953,279 bytes (100%) /opt/notebooks/Processed_final_split_1_16012024.tsvv
Processed 5000 rows for testing.
Analyzing Combination: prs_metabolomics
Analyzing Combination: pmh
Analyzing Combination: clinicalrisk
Analyzing Combination: everything
Separating boolean and numeric data...
Boolean columns: 447
Numeric columns: 236
Checking for missing or infinite values in numeric columns...
Removing low variance features...
Number of low variance features removed: 364
Number of features retained: 319
Removing highly correlated features from numeric columns...
Number of highly correlated features removed: 42
Number of features retained after correlation filtering: 277
Training model: RangerRF

Testing num_trees=50, max_depth=3...

Testing num_trees=50, max_depth=7...

Testing num_trees=50, max_depth=10...

Testing num_trees=100, max_depth=3...

Testing nu

In [ ]:
#hello